<a href="https://colab.research.google.com/github/thais-correia/automations/blob/main/Neighboring_Cities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neighboring Cities

In this case, I have a shapefile of Brazil's municipal limits (IBGE, 2021).

This automation aims to identify the neighboring limits of each of these cities.

For example, Brazil has 5,572 municipal boundaries.

One of these municipalities is São Paulo - SP.

The São Paulo city's neighboring limits are the cities of Caieiras, Guarulhos, Mairiporã, Osasco, Embu das Artes, among others.

So, what are the neighboring municipalities for each Brazilian city?

In [12]:
# Install necessary libraries
!pip install geopandas -q
!pip install tqdm -q

In [13]:
# Import the libraries
import geopandas as gpd
from tqdm import tqdm

In [14]:
# Here we set up access to our Google Drive
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [15]:
# Load the shapefile
shapefile_path = "/content/drive/MyDrive/Testes/BR_Municipios_2021.shp"
brazil = gpd.read_file(shapefile_path)

In [16]:
# Check the file type
type(brazil)

geopandas.geodataframe.GeoDataFrame

In [17]:
# View the first attributes of the geodataframe
brazil.head()

,CD_MUN,NM_MUN,SIGLA,AREA_KM2,geometry
0,1100015,Alta Floresta D'Oeste,RO,7067.127,"POLYGON ((-62.19465 -11.82746, -62.19332 -11.8..."
1,1100023,Ariquemes,RO,4426.571,"POLYGON ((-62.53648 -9.73222, -62.52765 -9.736..."
2,1100031,Cabixi,RO,1314.352,"POLYGON ((-60.37119 -13.36655, -60.37134 -13.3..."
3,1100049,Cacoal,RO,3793.000,"POLYGON ((-61.00080 -11.29737, -61.00103 -11.3..."
4,1100056,Cerejeiras,RO,2783.300,"POLYGON ((-61.49976 -13.00525, -61.49809 -13.0..."


In [18]:
# Create an empty column for neighboring polygons
brazil['neighbors'] = None

In [19]:
count=0
# Iterate over polygons and identify neighbors
for item, row in tqdm(brazil.iterrows()):
    try:
      neighbors = brazil[brazil.geometry.touches(row['geometry'])].NM_MUN.tolist()
      if len(neighbors) <=1:
         brazil.at[item, 'neighbors'] = neighbors[0]
      else:
         brazil.at[item, 'neighbors'] = ",".join(neighbors)
    except:
      print(row['NM_MUN'])
      brazil.at[item, 'neighbors'] = 'Error'
      count=count+1


1540it [01:13, 85.43it/s]

Fernando de Noronha


3506it [02:41, 40.70it/s]

Ilhabela


4607it [03:35, 10.08it/s]

Lagoa Mirim
Lagoa dos Patos


5572it [04:35, 20.20it/s]


In [23]:
# Count the number of lines (cities) and the number of columns (attributes)
brazil.shape

(5572, 6)

In [21]:
# View the result
brazil[['geometry', 'NM_MUN', 'neighbors']]

,geometry,NM_MUN,neighbors
0,"POLYGON ((-62.19465 -11.82746, -62.19332 -11.8...",Alta Floresta D'Oeste,"Nova Brasilândia D'Oeste,Rolim de Moura,Santa ..."
1,"POLYGON ((-62.53648 -9.73222, -62.52765 -9.736...",Ariquemes,"Jaru,Machadinho D'Oeste,Rio Crespo,Alto Paraís..."
2,"POLYGON ((-60.37119 -13.36655, -60.37134 -13.3...",Cabixi,"Colorado do Oeste,Pimenteiras do Oeste,Comodoro"
3,"POLYGON ((-61.00080 -11.29737, -61.00103 -11.3...",Cacoal,"Espigão D'Oeste,Pimenta Bueno,Presidente Médic..."
4,"POLYGON ((-61.49976 -13.00525, -61.49809 -13.0...",Cerejeiras,"Colorado do Oeste,Corumbiara,Pimenteiras do Oeste"
...,...,...,...
5567,"POLYGON ((-48.42801 -16.62789, -48.42803 -16.6...",Vianópolis,"Cristianópolis,Orizona,Pires do Rio,São Miguel..."
5568,"POLYGON ((-49.84740 -17.57796, -49.84719 -17.5...",Vicentinópolis,"Edealina,Edéia,Goiatuba,Joviânia,Pontalina,Por..."
5569,"POLYGON ((-47.10998 -14.67195, -47.10994 -14.6...",Vila Boa,"Buritis,Flores de Goiás,Formosa"
5570,"POLYGON ((-48.75572 -14.86393, -48.75502 -14.8...",Vila Propício,"Barro Alto,Cocalzinho de Goiás,Goianésia,Mimos..."


In [22]:
# Export result to shapefile
brazil.to_file("/content/drive/MyDrive/Testes/neighboring_cities.shp")

**Thank you for your attention!** 🌎